Você pode adquirir versões impressas e de e-book do *Think Python 3e* (em inglês) em
[Bookshop.org](https://bookshop.org/a/98697/9781098155438) e
[Amazon](https://www.amazon.com/_/dp/1098155432?smid=ATVPDKIKX0DER&_encoding=UTF8&tag=oreilly20-20&_encoding=UTF8&tag=greenteapre01-20&linkCode=ur2&linkId=e2a529f94920295d27ec8a06e757dc7c&camp=1789&creative=9325).

Uma versão em língua portuguesa da 3ª edição foi publicada pela editora [Novatec](https://novatec.com.br/livros/pense-em-python-3ed/).

In [ ]:
from os.path import basename, exists

def download(url):
    filename = basename(url)
    if not exists(filename):
        from urllib.request import urlretrieve

        local, _ = urlretrieve(url, filename)
        print("Downloaded " + str(local))
    return filename

download('https://github.com/AllenDowney/ThinkPython/raw/v3/thinkpython.py');
download('https://github.com/AllenDowney/ThinkPython/raw/v3/diagram.py');

import thinkpython

# Análise e geração de texto

Neste ponto, cobrimos as principais estruturas de dados do Python -- listas, dicionários e tuplas -- e alguns algoritmos que as usam.
Neste capítulo, usaremos essas estruturas para explorar a análise de texto e a geração de Markov:

* A análise de texto é uma maneira de descrever as relações estatísticas entre as palavras em um documento, como a probabilidade de uma palavra ser seguida por outra, e

* A geração de Markov é uma maneira de gerar um novo texto com palavras e frases semelhantes ao texto original.

Esses algoritmos são semelhantes a partes de uma *Large Language Model* (LLM), que é o principal componente de um chatbot.

Começaremos contando o número de vezes que cada palavra aparece em um livro.
Depois, veremos pares de palavras e faremos uma lista das palavras que podem vir depois de cada palavra.
Faremos uma versão simples de um gerador de Markov e, como exercício, você terá a chance de fazer uma versão mais geral.

## Palavras únicas

Como um primeiro passo em direção à análise de texto, vamos ler um livro -- *The Strange Case Of Dr. Jekyll And Mr. Hyde* de Robert Louis Stevenson -- e contar o número de palavras únicas.
As instruções para baixar o livro estão no notebook deste capítulo.

A célula a seguir baixa o livro do Projeto Gutenberg:

In [ ]:
download('https://www.gutenberg.org/cache/epub/43/pg43.txt');

A versão disponível no Project Gutenberg inclui informações sobre o livro no início e informações sobre a licença no final.
Usaremos `clean_file` do Capítulo 8 para remover esse material e escrever um arquivo "clean" que contenha apenas o texto do livro.

Usaremos um laço `for` para ler linhas do arquivo e `split` para separar as linhas em palavras.
Então, para manter o controle de palavras únicas, armazenaremos cada palavra como uma chave em um dicionário:

O comprimento do dicionário é o número de palavras únicas -- cerca de `6000` por esta forma de contagem.
Mas se as inspecionarmos, veremos que algumas não são palavras válidas.

Por exemplo, vamos olhar as palavras mais longas em `unique_words`.
Podemos usar `sorted` para ordenar as palavras, passando a função `len` como um argumento de palavra-chave para que as palavras sejam classificadas por comprimento:

O índice de fatiamento, `[-5:]`, seleciona os últimos `5` elementos da lista ordenada, que são as palavras mais longas.

A lista inclui algumas palavras legitimamente longas, como "*circumscription*", e algumas palavras hifenizadas, como "*chocolate-coloured*".
Mas algumas das "palavras" mais longas são, na verdade, duas palavras separadas por um traço.
E outras palavras incluem pontuação como pontos, pontos de exclamação e aspas.

Então, antes de prosseguirmos, vamos lidar com traços e outras pontuações.

## Pontuação

Para identificar as palavras no texto, precisamos lidar com dois problemas:

* Quando um traço aparece em uma linha, devemos substituí-lo por um espaço -- assim, quando usamos `split`, as palavras serão separadas.

* Depois de dividir as palavras, podemos usar `strip` para remover a pontuação.

Para lidar com o primeiro problema, podemos usar a seguinte função, que recebe uma *string*, substitui traços por espaços, separa a *string* e retorna a lista resultante:

Observe que `split_line` substitui apenas travessões, não hifens.
Aqui está um exemplo:

Agora, para remover a pontuação do início e do fim de cada palavra, podemos usar `strip`, mas precisamos de uma lista de caracteres que são considerados pontuação.

Os caracteres em *strings* Python estão em Unicode, que é um padrão internacional usado para representar letras em quase todos os alfabetos, números, símbolos, sinais de pontuação e muito mais.
O módulo `unicodedata` fornece uma função `category` que podemos usar para dizer quais caracteres são pontuação.
Dada uma letra, ele devolve uma *string* com informações sobre em qual categoria a letra está:

A categoria da *string* `'A'` é `'Lu'` -- o `'L'` significa que é uma letra e o `'u'` significa que é maiúscula.

A categoria da *string* `'.'` é `'Po'` -- o `'P'` significa que é pontuação e o `'o'` significa que sua subcategoria é "outro".

Podemos encontrar os sinais de pontuação no livro verificando caracteres de categorias que começam com `'P'`.
O laço de repetição a seguir armazena os sinais de pontuação únicos em um dicionário:

Para fazer uma lista de sinais de pontuação, podemos juntar as chaves do dicionário em uma *string*:

Agora que sabemos quais caracteres no livro são pontuação, podemos escrever uma função que recebe uma palavra, retira a pontuação do começo e do fim e a converte para minúsculas:

Aqui está um exemplo:

Como `strip` remove caracteres do início e do fim, ela deixa as palavras hifenizadas intactas:

Agora, aqui está um laço de repetição que usa `split_line` e `clean_word` para identificar as palavras únicas no livro:

Com essa definição mais rigorosa do que é uma palavra, há cerca de 4000 palavras únicas.
E podemos confirmar que a lista de palavras mais longas foi limpa:

Agora vamos ver quantas vezes cada palavra é usada.

## Frequências de palavras

O laço de repetição a seguir calcula a frequência de cada palavra única:

A primeira vez que vemos uma palavra, inicializamos sua frequência em `1`. Se virmos a mesma palavra novamente mais tarde, incrementamos sua frequência.

Para ver quais palavras aparecem com mais frequência, podemos usar `items` para obter os pares chave-valor de `word_counter` e ordená-los pelo segundo elemento do par, que é a frequência.
Primeiro, definiremos uma função que seleciona o segundo elemento:

Now we can use `sorted` with two keyword arguments:

* `key=second_element` means the items will be sorted according to the frequencies of the words.

* `reverse=True` means they items will be sorted in reverse order, with the most frequent words first.

Agora podemos usar `sorted` com dois argumentos de palavra-chave:

* `key=second_element` significa que os itens serão ordenados de acordo com as frequências das palavras.

* `reverse=True` significa que os itens serão ordenados em ordem reversa, com as palavras mais frequentes primeiro.

Aqui estão as cinco palavras mais frequentes:

Na próxima seção, encapsularemos esse laço de repetição em uma função.
E o usaremos para demonstrar um novo recurso -- parâmetros opcionais.

## Parâmetros opcionais

Usamos funções internas que aceitam parâmetros opcionais.
Por exemplo, `round` aceita um parâmetro opcional chamado `ndigits` que indica quantas casas decimais manter:

Mas não são apenas as funções internas -- podemos escrever funções com parâmetros opcionais também.
Por exemplo, a função a seguir recebe dois parâmetros, `word_counter` e `num`:

O segundo parâmetro parece uma instrução de atribuição, mas não é -- é um parâmetro opcional.

Se você chamar esta função com um argumento, `num` recebe o **valor padrão**, que é `5`:

Se você chamar esta função com dois argumentos, o segundo argumento será atribuído a `num` em vez do valor padrão:

Nesse caso, diríamos que o argumento opcional **sobrescreve** o valor padrão.

Se uma função tiver parâmetros obrigatórios e opcionais, todos os parâmetros obrigatórios devem vir primeiro, seguidos pelos opcionais:

In [ ]:
%%expect SyntaxError


## Subtração de dicionário

Suponha que queremos verificar a ortografia de um livro -- isto é, encontrar uma lista de palavras que podem estar escritas incorretamente.
Uma maneira de fazer isso é encontrar palavras no livro que não aparecem em uma lista de palavras válidas.
Em capítulos anteriores, usamos uma lista de palavras que são consideradas válidas em jogos de palavras como Scrabble.
Agora usaremos essa lista para verificar a ortografia de Robert Louis Stevenson.

Podemos pensar nesse problema como uma subtração de conjunto -- isto é, queremos encontrar todas as palavras de um conjunto (as palavras no livro) que não estão no outro (as palavras na lista).

A célula a seguir baixa a lista de palavras:

In [ ]:
download('https://raw.githubusercontent.com/AllenDowney/ThinkPython/v3/words.txt');

Como fizemos antes, podemos ler o conteúdo de `words.txt` e separá-lo em uma lista de *strings*:

Armazenaremos as palavras como chaves em um dicionário para que possamos usar o operador `in` para verificar rapidamente se uma palavra é válida:

Agora, para identificar palavras que aparecem no livro, mas não na lista de palavras, usaremos `subtract`, que recebe dois dicionários como parâmetros e devolve um novo dicionário que contém todas as chaves de um que não estão no outro:

Veja como a usamos:

Para obter uma amostra de palavras que podem estar escritas incorretamente, podemos exibir as palavras mais comuns em `diff`:

As palavras "com erros ortográficos" mais comuns são principalmente nomes e algumas palavras de uma única letra (o Sr. Utterson é amigo e advogado do Dr. Jekyll) (*Mr. Utterson is Dr. Jekyll's friend and lawyer*).

Se selecionarmos palavras que aparecem apenas uma vez, é mais provável que sejam erros ortográficos reais.
Podemos fazer isso percorrendo os itens e criando uma lista de palavras com frequência `1`:

Aqui estão os últimos elementos da lista:

A maioria delas são palavras válidas que não estão na lista de palavras.
Mas `'reindue'` parece ser um erro de grafia de `'reinduce'`, então pelo menos encontramos um erro legítimo.

## Números aleatórios

Como um passo em direção à geração de texto de Markov, a seguir escolheremos uma sequência aleatória de palavras de `word_counter`.
Mas primeiro vamos falar sobre aleatoriedade.

Dadas as mesmas entradas, a maioria dos programas de computador são **determinísticos**, o que significa que geram as mesmas saídas todas as vezes.
O determinismo geralmente é uma coisa boa, pois esperamos que o mesmo cálculo produza o mesmo resultado.
Para algumas aplicações, no entanto, queremos que o computador seja imprevisível.
Jogos são um exemplo, mas há mais.

Tornar um programa verdadeiramente não determinístico acaba sendo difícil, mas há maneiras de fingir.
Uma é usar algoritmos que geram números **pseudoaleatórios**.
Números pseudoaleatórios não são verdadeiramente aleatórios porque são gerados por uma computação determinística, mas apenas olhando para os números é quase impossível distingui-los do aleatório.

O módulo `random` fornece funções que geram números pseudoaleatórios -- que eu simplesmente chamarei de "aleatórios" daqui em diante.
Podemos importá-lo assim:

In [ ]:
  # esta célula inicializa o gerador de números aleatórios
  # para que ele gere a mesma sequência toda vez que o
  # notebook é executado.

random.seed(4)

O módulo `random` fornece uma função chamada `choice` que escolhe aleatoriamente um elemento de uma lista, com cada elemento tendo a mesma probabilidade de ser escolhido:

Se você chamar a função novamente, poderá obter o mesmo elemento novamente ou um diferente:

A longo prazo, esperamos obter cada elemento aproximadamente o mesmo número de vezes.

Se você usar `choice` com um dicionário, obterá um `KeyError`:

In [ ]:
%%expect KeyError



Para escolher uma chave aleatória, você deve colocar as chaves em uma lista e então chamar `choice`:

Se gerarmos uma sequência aleatória de palavras, não fará muito sentido:

Parte do problema é que não estamos levando em conta que algumas palavras são mais comuns do que outras.
Os resultados serão melhores se escolhermos palavras com "pesos" diferentes, para que algumas sejam escolhidas com mais frequência do que outras.

Se usarmos os valores de `word_counter` como pesos, cada palavra será escolhida com uma probabilidade que depende de sua frequência:

O módulo `random` fornece outra função chamada `choices` que recebe pesos como um argumento opcional:

E recebe outro argumento opcional, `k`, que especifica o número de palavras a serem selecionadas:

O resultado é uma lista de *strings* que podemos juntar em algo que se parece mais com uma frase:

Se você escolher palavras do livro aleatoriamente, você fica com uma noção do vocabulário, mas uma série de palavras aleatórias raramente faz sentido porque não há relação entre palavras sucessivas.
Por exemplo, em uma frase real você espera que um artigo como "the" seja seguido por um adjetivo ou substantivo, e provavelmente não um verbo ou advérbio.
Então o próximo passo é olhar para essas relações entre palavras:

## Bigramas

Em vez de olhar para uma palavra de cada vez, agora olharemos para sequências de duas palavras, que são chamadas de **bigramas**.
Uma sequência de três palavras é chamada de **trigrama**, e uma sequência com um número não especificado de palavras é chamada de **n-grama**.

Vamos escrever um programa que encontra todos os bigramas no livro e o número de vezes que cada um aparece.
Para armazenar os resultados, usaremos um dicionário em que

* As chaves são tuplas de *strings* que representam bigramas, e

* Os valores são inteiros que representam frequências.

Vamos chamá-lo de `bigram_counter`:

A função a seguir recebe uma lista de duas *strings* como parâmetro.
Primeiro, ela faz uma tupla das duas *strings*, que pode ser usada como uma chave em um dicionário.
Então, ela adiciona a chave a `bigram_counter`, se ela não existir, ou incrementa a frequência se existir:

À medida que avançamos no livro, temos que manter o registro de cada par de palavras consecutivas.
Então, se virmos a sequência "*man is not truly one*", adicionaríamos os bigramas "*man is*", "*is not*", "*not truly*" e assim por diante.

Para manter o registro desses bigramas, usaremos uma lista chamada `window`, porque é como uma janela que desliza sobre as páginas do livro, mostrando apenas duas palavras por vez.
Inicialmente, `window` está vazia:

Usaremos a seguinte função para processar as palavras uma de cada vez:

A primeira vez que essa função é chamada, ela acrescenta a palavra dada a `window`.
Como há apenas uma palavra na janela, ainda não temos um bigrama, então a função termina.

A segunda vez que é chamada -- e todas as vezes depois disso -- ela acrescenta uma segunda palavra a `window`.
Como há duas palavras na janela, ela chama `count_bigram` para manter o registro de quantas vezes cada bigrama aparece.
Então, ela usa `pop` para remover a primeira palavra da janela.

O programa a seguir percorre as palavras no livro e as processa uma de cada vez:

O resultado é um dicionário que mapeia cada bigrama para o número de vezes que ele aparece.
Podemos usar `print_most_common` para ver os bigramas mais comuns:

Olhando para esses resultados, podemos ter uma ideia de quais pares de palavras têm mais probabilidade de aparecerem juntas.
Também podemos usar os resultados para gerar texto aleatório, como este:

`bigrams` é uma lista dos bigramas que aparecem nos livros.
`weights` é uma lista de suas frequências, então `random_bigrams` é uma amostra onde a probabilidade de um bigrama ser selecionado é proporcional à sua frequência.

Aqui estão os resultados:

Essa maneira de gerar texto é melhor do que escolher palavras aleatórias, mas ainda não faz muito sentido.

## Análise de Markov

Podemos fazer melhor com a análise de texto de cadeia de Markov, que computa, para cada palavra em um texto, a lista de palavras que vêm a seguir.
Como exemplo, analisaremos estas letras da música *Eric, the Half a Bee* do Monty Python:

In [ ]:
song = """
Half a bee, philosophically,
Must, ipso facto, half not be.
But half the bee has got to be
Vis a vis, its entity. D'you see?
"""

To store the results, we'll use a dictionary that maps from each word to the list of words that follow it.
Para armazenar os resultados, usaremos um dicionário que mapeia cada palavra para a lista de palavras que a segue:

Como exemplo, vamos começar com as duas primeiras palavras da música:

Se a primeira palavra não estiver em `successor_map`, temos que adicionar um novo item que mapeie a primeira palavra para uma lista contendo a segunda palavra:

Se a primeira palavra já estiver no dicionário, podemos procurá-la para obter a lista de sucessoras que vimos até agora e acrescentar a nova:

A função a seguir encapsula essas etapas:

Se o mesmo bigrama aparecer mais de uma vez, a segunda palavra será adicionada à lista mais de uma vez.
Dessa forma, `successor_map` mantém o registro de quantas vezes cada sucessor aparece.

Como fizemos na seção anterior, usaremos uma lista chamada `window` para armazenar pares de palavras consecutivas.
E usaremos a seguinte função para processar as palavras uma de cada vez:

Veja como a usamos para processar as palavras na música.

E aqui estão os resultados:

A palavra `'half'` pode ser seguida por `'a'`, `'not'` ou `'the'`.
A palavra `'a'` pode ser seguida por `'bee'` ou `'vis'`.
A maioria das outras palavras aparece apenas uma vez, então elas são seguidas por apenas uma única palavra.

Agora vamos analisar o livro:

Podemos procurar qualquer palavra e encontrar as palavras que podem vir depois dela:

In [ ]:
# Usei esta célula para encontrar um predecessor com um bom número de possíveis
# sucessores e pelo menos uma palavra repetida.



Nesta lista de sucessores, observe que a palavra `'to'` aparece três vezes -- os outros sucessores aparecem apenas uma vez.

## Gerando texto

Podemos usar os resultados da seção anterior para gerar um novo texto com as mesmas relações entre palavras consecutivas do original.
Veja como funciona:

* Começando com qualquer palavra que apareça no texto, procuramos suas possíveis sucessoras e escolhemos uma aleatoriamente.

* Então, usando a palavra escolhida, procuramos suas possíveis sucessoras e escolhemos uma aleatoriamente.

Podemos repetir esse processo para gerar quantas palavras quisermos.
Como exemplo, vamos começar com a palavra `'embora'`.
Aqui estão as palavras que podem vir depois dela:

In [ ]:
# esta célula inicializa o gerador de números aleatórios
# para que ele comece no mesmo ponto da sequência toda
# vez que este notebook for executado.

random.seed(2)

Podemos usar `choice` para escolher na lista com a mesma probabilidade:

Se a mesma palavra aparecer mais de uma vez na lista, é mais provável que ela seja selecionada.

Repetindo esses passos, podemos usar o seguinte laço de reptição para gerar uma série mais longa:

O resultado parece mais com uma frase real, mas ainda não faz muito sentido.

Podemos fazer melhor usando mais de uma palavra como chave em `successor_map`.
Por exemplo, podemos fazer um dicionário que mapeia de cada bigrama -- ou trigrama -- para a lista de palavras que vem a seguir.
Como exercício, você terá a chance de implementar essa análise e ver como ficam os resultados.

## Depuração

Neste ponto, estamos escrevendo programas mais substanciais, e você pode se dar conta de que está gastando mais tempo depurando.
Se você estiver preso em um bug difícil, aqui estão algumas coisas para tentar:

* Leitura: Examine seu código, leia-o novamente para si mesmo e verifique se ele diz o que você quis dizer.

* Execução: Experimente fazer alterações e executar versões diferentes. Muitas vezes, se você exibir a coisa certa no lugar certo no programa, o problema se tornará óbvio, mas às vezes você terá que construir códigos de suporte.

* Ruminando: Reserve um tempo para pensar! Que tipo de erro é: sintaxe, tempo de execução, ou semântica? Que informações você pode obter das mensagens de erro, ou da saída do programa? Que tipo de erro pode causar o problema que você está vendo? O que você mudou por último, antes do
problema aparecer?

* *Rubberducking*: Se você explicar o problema para outra pessoa, às vezes você encontra a resposta antes de terminar de fazer a pergunta. Muitas vezes você não precisa da outra pessoa; você poderia simplesmente falar com um pato de borracha. E essa é a origem da estratégia bem conhecida chamada **depuração do pato de borracha**. Não estou inventando isso -- veja <https://pt.wikipedia.org/wiki/Debug_com_Pato_de_Borracha> (em inglês, <https://en.wikipedia.org/wiki/Rubber_duck_debugging>).

* Recuando: Em algum momento, a melhor coisa a fazer é dar um passo atrás -- desfazendo as alterações recentes -- até chegar a um programa que funcione. Então você pode começar a reconstruir.

* Descansando: Se você der um descanso ao seu cérebro, em algum momento ele encontrará o problema para você.

Às vezes, programadores iniciantes ficam presos em uma dessas atividades e esquecem as outras. Cada atividade vem com seu próprio modo de falha.

Por exemplo, ler seu código funciona se o problema for um erro tipográfico, mas não se o problema for um mal-entendido conceitual.
Se você não entende o que seu programa faz, pode lê-lo 100 vezes e nunca ver o erro, porque o erro está na sua cabeça.

Executar experimentos pode funcionar, especialmente se você executar testes pequenos e simples.
Mas se você executar experimentos sem pensar ou ler seu código, pode levar muito tempo para descobrir o que está acontecendo.

Você tem que reservar um tempo para pensar. A depuração é como uma ciência experimental. Você deve ter pelo menos uma hipótese sobre qual é o problema. Se houver duas ou mais possibilidades, tente pensar em um teste que elimine uma delas.

Mas mesmo as melhores técnicas de depuração falharão se houver muitos
erros, ou se o código que você está tentando consertar for muito grande e complicado.
Às vezes, a melhor opção é recuar, simplificando o programa até
você voltar a algo que funcione.

Programadores iniciantes geralmente relutam em recuar porque não
suportam excluir uma linha de código (mesmo que esteja errada). Se isso fizer você se sentir melhor, copie seu programa para outro arquivo antes de começar a desfazê-lo. Então você pode copiar as partes de volta, uma de cada vez.

Encontrar um bug difícil requer leitura, execução, ruminação, recuo e, às vezes, descanso.
Se você ficar preso em uma dessas atividades, tente as outras.

## Glossário

**valor padrão** (*default value)**:**
O valor atribuído a um parâmetro se nenhum argumento for fornecido.

**sobrescrever** (*override*)**:**
Substituir um valor padrão por um argumento.

**determinístico** (*deterministic*)**:**
Um programa determinístico faz a mesma coisa toda vez que é executado, dadas as mesmas entradas.

**pseudoaleatório** (*pseudorandom*)**:**
Uma sequência pseudoaleatória de números parece ser aleatória, mas é gerada por um programa determinístico.

**bigrama** (*bigram*)**:**
Uma sequência de dois elementos, geralmente palavras.

**trigrama** (*trigram*)**:**
Uma sequência de três elementos.

**n-grama** (*n-gram*)**:**
Uma sequência de um número não especificado de elementos.

**depuração do pato de borracha** (*rubber duck debugging*)**:**
Uma maneira de depurar explicando um problema em voz alta para um objeto inanimado.

## Exercícios

In [ ]:
# Esta célula diz ao Jupyter para fornecer informações detalhadas de depuração
# quando ocorre um erro de tempo de execução. Execute-a antes de trabalhar nos
# exercícios.

%xmode Verbose

### Pergunte a um assistente virtual

Em `add_bigram`, a instrução `if` cria uma nova lista ou acrescenta um elemento a uma lista existente, dependendo se a chave já está no dicionário:

In [ ]:
def add_bigram(bigram):
    first, second = bigram

    if first not in successor_map:
        successor_map[first] = [second]
    else:
        successor_map[first].append(second)

Os dicionários possuem um método chamado `setdefault` que podemos usar para fazer a mesma coisa de forma mais concisa.
Pergunte a um assistente virtual como ele funciona ou copie `add_word` para um assistente virtual e pergunte "Você pode reescrever isso usando `setdefault`?" ("*Can you rewrite this using `setdefault`?*").

Neste capítulo, implementamos a análise e geração de texto em cadeia de Markov.
Se você estiver curioso, pode pedir mais informações sobre o tópico a um assistente virtual.
Uma das coisas que você pode aprender é que os assistentes virtuais usam algoritmos que são semelhantes em muitos aspectos -- mas também diferentes em aspectos importantes.
Pergunte a um assistente virtual: "Quais são as diferenças entre grandes modelos de linguagem como GPT e análise de texto em cadeia de Markov?" ("*What are the differences between large language models like GPT and Markov chain text analysis?*")



### Exercício

Escreva uma função que conte o número de vezes que cada trigrama (sequência de três palavras) aparece.
Se você testar sua função com o texto de _Dr. Jekyll e Mr. Hyde_, você deve descobrir que o trigrama mais comum é "said the lawyer".

Dica: Escreva uma função chamada `count_trigram` que seja semelhante a `count_bigram`. Em seguida, escreva uma função chamada `process_word_trigram` que seja semelhante a `process_word_bigram`.

In [ ]:
# A solução vai aqui

In [ ]:
# A solução vai aqui

Você pode usar o seguinte laço de repetição para ler o livro e processar as palavras:

In [ ]:
trigram_counter = {}
window = []

for line in open(filename):
    for word in split_line(line):
        word = clean_word(word)
        process_word_trigram(word)

Em seguida, use `print_most_common` para encontrar os trigramas mais comuns no livro:

In [ ]:
print_most_common(trigram_counter)

### Exercício

Agora vamos implementar a análise de texto da cadeia de Markov com um mapeamento de cada bigrama para uma lista de possíveis sucessores.

Começando com `add_bigram`, escreva uma função chamada `add_trigram` que recebe uma lista de três palavras e adiciona ou atualiza um item em `successor_map`, usando as duas primeiras palavras como a chave e a terceira palavra como uma possível sucessora.

In [ ]:
# A solução vai aqui

Aqui está uma versão de `process_word_trigram` que chama `add_trigram`:

In [ ]:
def process_word_trigram(word):
    window.append(word)

    if len(window) == 3:
        add_trigram(window)
        window.pop(0)

Você pode usar o seguinte laço de repetição para testar sua função com a letra de "Eric, the Half a Bee":

In [ ]:
successor_map = {}
window = []

for string in song.split():
    word = string.strip(punctuation).lower()
    process_word_trigram(word)

Se sua função funcionar como pretendido, o predecessor `('half', 'a')` deve mapear para uma lista com o único elemento `'bee'`.
Na verdade, como acontece, cada bigrama nesta música aparece apenas uma vez, então todos os valores em `successor_map` têm um único elemento:

In [ ]:
successor_map

Você pode usar o seguinte laço de repetição para testar sua função com as palavras do livro:

In [ ]:
successor_map = {}
window = []

for line in open(filename):
    for word in split_line(line):
        word = clean_word(word)
        process_word_trigram(word)

No próximo exercício, você usará os resultados para gerar um novo texto aleatório.

### Exercício

Para este exercício, assumiremos que `successor_map` é um dicionário que mapeia cada bigrama para a lista de palavras que aparecem depois dele.

In [ ]:
# esta célula inicializa o gerador de números aleatórios
# para que ele comece no mesmo ponto da sequência toda
# vez que este notebook for executado.

random.seed(3)

Para gerar texto aleatório, começaremos escolhendo uma chave aleatória de `successor_map`:

In [ ]:
successors = list(successor_map)
bigram = random.choice(successors)
bigram

Agora escreva um laço de repetiçaão que gere mais 50 palavras seguindo estas etapas:

1. Em `successor_map`, procure a lista de palavras que podem vir depois de um `bigram`.

2. Escolha uma delas aleatoriamente e exiba na tela.

3. Para a próxima iteração, faça um novo bigrama que contenha a segunda palavra de `bigram` e a sucessor escolhido.

Por exemplo, se começarmos com o bigrama `('duvidou', 'se')` e escolhermos `'de'` como seu sucessor, o próximo bigrama será `('se', 'de')`.

In [ ]:
# A solução vai aqui

Se tudo estiver funcionando, você deve perceber que o texto gerado é reconhecidamente similar em estilo ao original, e algumas frases fazem sentido, mas o texto pode desviar de um tópico para outro.

Como um exercício bônus, modifique sua solução para os dois últimos exercícios para usar trigramas como chaves em `successor_map`, e veja qual efeito isso tem nos resultados.

[Pense Python: 3ª Edição](https://rodrigocarlson.github.io/PensePython3ed/)

Copyright 2024 [Allen B. Downey](https://allendowney.com/) (versão original)

Copyright 2025 [Rodrigo Castelan Carlson](https://rodrigocarlson.paginas.ufsc.br/) (desta versão)

Foram preservadas as mesmas licenças da versão original.

Licença dos códigos: [MIT License](https://mit-license.org/)

Licença dos textos: [Creative Commons Attribution-NonCommercial-ShareAlike 4.0 International](https://creativecommons.org/licenses/by-nc-sa/4.0/)